In [1]:
print("ok")

ok


In [2]:
%pwd

'e:\\Medibot\\research'

In [3]:
import os
os.chdir("../")

In [1]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [2]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [3]:
extracted_data=load_pdf_file(data='E:\Medibot\Data')

In [4]:
def text_split(extracted_data):
    text_spiltter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spiltter.split_documents(extracted_data)
    return text_chunks

In [5]:
text_chunks=text_split(extracted_data)
print("Length of Text chunks",len(text_chunks))

Length of Text chunks 5860


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

In [7]:
def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [8]:
embeddings=download_hugging_face_embedding()

C:\Users\SMTA\AppData\Local\Temp\ipykernel_28764\2320044209.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
e:\Medibot\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
query_result=embeddings.embed_query('apple')
print('Length',len(query_result))

Length 384


In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec

pc=pinecone(api_key=PINECONE_API_KEY)
index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-qrim1iv.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [11]:

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [12]:
from langchain_pinecone import PineconeVectorStore
index_name="medibot"
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [18]:
docsearch

In [13]:
retriever = docsearch.as_retriever(search_type='similarity',search_kwargs={"k":3})

In [14]:
retrieved_docs= retriever.invoke("what s Acne?") 

In [21]:
retrieved_docs

[Document(id='c01b25b9-268c-41e7-a9a9-673c93819b9d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'E:\\Medibot\\Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed (1).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='89dd4c6f-af27-440e-a81a-6949340770d9', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'E:\\Medibot\\Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed (1).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become infla

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers

In [14]:
prompt_template=""" 
You are a helpful, knowledgeable, and responsible SMTA medical AI assistant.  
You are not a doctor, but you provide medically accurate information, help users understand symptoms, explain medical conditions in simple terms, and guide them to seek professional help when necessary.  
If you don't know the answer, simply say that you don't know. Do not attempt to make up an answer.  
Context: {context}  
Question: {question}  

For serious or life-threatening conditions, recommend that the user seek immediate medical attention.

Always recommend users consult with a licensed healthcare provider for diagnosis and treatment.

Helpful answer:

"""

In [15]:
PROMPT = PromptTemplate(template=prompt_template , input_variables=["context","question"])
chain_type_kwargs ={"prompt":PROMPT}

In [16]:
llm=CTransformers(model="E:\Medibot\Model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  confiq={'max_new_tokens': 512,
                        'temperature': 0.8
                         })

In [17]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [18]:


while True:
    user_input = input("Input prompt: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    result = qa.invoke({"query": user_input})  # Direct call without threading
    print("\n🧠 Response:\n", result["result"])




🧠 Response:
 Acne is a common skin condition that affects people of all ages, but it's most common among teenagers. It occurs when the pores on your skin become clogged with oil, dead skin cells, and bacteria. This can lead to inflammation and cause redness, swelling, and pus-filled bumps on the skin.
There are several types of acne, including blackheads, whiteheads, and papules. Blackheads are small, dark spots that appear on the skin when oil and dead skin cells clog the pores. Whiteheads are similar to blackheads but don't have a visible tip. Papules are small, pink bumps that can be tender to the touch.
Acne can be caused by several factors, including hormonal changes, genetabolism (menstruggles, geneturns changes, environmental changes, genetic changes, genet, genet changes, genet changes, genet alters, genetss, genetabolism, geneturns changes, geneternal changes, genes, environmental and genetastic changes, genetabolance, environmental triggers (estrogenres

🧠 Response:
 A brain

MaxRetryError: HTTPSConnectionPool(host='medibot-qrim1iv.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000027C7D928A30>: Failed to resolve 'medibot-qrim1iv.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))